In [1]:
import pandas as pd
import numpy as np
import copy
#import emoji

#make the columns as wide as possible so we can see all the text
pd.set_option('display.max_colwidth', None)

In [2]:
original_training = pd.read_csv("mediaeval-2015-trainingset.txt", delimiter = "\t")

#drop all columns apart from the text and the label as none of the other data appears to be useful
original_training = original_training.drop(["tweetId", "userId", "imageId(s)", "username", "timestamp"], axis = 1)

#add a column to store the language
original_training["lang"] = np.nan

#view a specific range, uncomment to view only those tweets from certain classes
original_training[10000:10020]

,tweetText,label,lang
10000,So touching! RT @DreamCameTrue_: RT @Alyssa_Milano: Thank goodness for people who are kind #sandy http://t.co/yBaVo3FZ /via @CarrieFairygirl,real,NaN
10001,Thank goodness for people who are kind #sandy http://t.co/Pc25SgSy /via @CarrieFairygirl,real,NaN
10002,RT “@Alyssa_Milano: Thank goodness for people who are kind #sandy http://t.co/wfMtqwjl /via @CarrieFairygirl”\n\nComplete fire hazard,real,NaN
10003,RT “@Alyssa_Milano: Thank goodness for people who are kind #sandy http://t.co/wfMtqwjl /via @CarrieFairygirl”\n\nComplete fire hazard,real,NaN
10004,Thank goodness for people who are kind #sandy http://t.co/ghz8E5je /via @CarrieFairygirl,real,NaN
10005,@Alyssa_Milano: Thank goodness for people who are kind #sandy http://t.co/WE72RVCE /via @CarrieFairygirl LOVE THIS!,real,NaN
10006,RT @alyssa_milano: Thank goodness for people who are kind #sandy http://t.co/45y8vlMQ /via @CarrieFairygirl,real,NaN
10007,“@Alyssa_Milano: Thank goodness for people who are kind #sandy http://t.co/wuZ5PyNk /via @CarrieFairygirl” Awesome.,real,NaN
10008,RT @Alyssa_Milano Thank goodness for people who are kind #sandy http://t.co/3g21wFvG /via @CarrieFairygirl,real,NaN
10009,"É, a humanidade ainda tem jeito. “@Alyssa_Milano: Thank goodness for people who are kind #sandy http://t.co/xwu5jSIg /via @CarrieFairygirl”",real,NaN


In [3]:
#repeat the same process for the testing dataset
original_testing = pd.read_csv("mediaeval-2015-testset.txt", delimiter = "\t")
original_testing = original_testing.drop(["tweetId", "userId", "imageId(s)", "username", "timestamp"], axis = 1)
original_testing["lang"] = np.nan

In [4]:
#we can see that the dataset is skewed towards fake and humor tweets
original_training.label.value_counts()

fake     6742
real     4921
humor    2614
Name: label, dtype: int64

In [5]:
#there are no non null values to being with we can see
original_training.info()
print("\n")
original_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14277 entries, 0 to 14276
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweetText  14277 non-null  object 
 1   label      14277 non-null  object 
 2   lang       0 non-null      float64
dtypes: float64(1), object(2)
memory usage: 334.7+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweetText  3755 non-null   object 
 1   label      3755 non-null   object 
 2   lang       0 non-null      float64
dtypes: float64(1), object(2)
memory usage: 88.1+ KB


In [6]:
import langdetect as l
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import nltk.stem as st
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\George\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\George\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
#view the languages supported by the stemming algorithm
st.SnowballStemmer.languages

('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

In [8]:
#responsible for parsing tweets
class TweetHandler:
    
    def __init__(self):
        snowball_langs = list(st.SnowballStemmer.languages)
        self.tokenizer_langs = {"da", "nl", "en", "fi", "fr", "de", "it", "pt", "ru", "es", "sv"}
        langdetect_langs = ["ar", "da", "nl", "en", "fi", "fr", "de", "hu", "it", "no", "illegal", "pt", "ro", "ru", "es", "sv"]
        #a dictionary to map the corresponding snowball and langdetect properties
        self.lang_dict = dict(zip(langdetect_langs, snowball_langs))
        #declare some custom stop words
        self.custom_stops = ["http","nhttp","https"]

    #takes a tweet, detects its language, removes any stop words in the language,
    #stems tokens in language and returns the simplified tokens paired with the language
    def parse_tweet(self, tweet):
        
        try:
            lang_prediction = l.detect(tweet)
            #the nltk name for the predicted language
            nltkprop = self.lang_dict[lang_prediction]
        except:
            #assume english stopwords and stemming if the language cannot be detected
            lang_prediction = "unknown"
            nltkprop = "english"
            
        #if the language is not supported by the tokenizer (including unkown) then assume tokenizing in English, however stemming
        #and stopwords may still be supported in the language e.g. arabic, hungarian, romanian so tokenize with the english
        #version of the algorithm if this is the case and use the stemming and stopwords specific to the language if this
        #is available even if the tokenization algorithm isnt
        tokens = nltk.word_tokenize(tweet, language = nltkprop if lang_prediction in self.tokenizer_langs else "english")
        
        #stop words specific to the language
        stop_words = set(stopwords.words(nltkprop))
        
        #stemming algorithm specific to the language detected
        stemmer = st.SnowballStemmer(nltkprop)
        
        #store all tokens to be output here, filter out any unwanted tokens and don't add them
        #store the tokens in a space seperated string so that it can be fed into a count vectorizer
        filtered_tokens = ""
        
        for tok in tokens:
            
            #remove any hashtags
            if tok[0] == '#':
                tok = tok[1:]
                
            #discard non alphanumeric strings containing symbols or pure digits, or stop words
            if (not tok.isalnum()) or tok.isdigit() or (tok in stop_words) or tok in self.custom_stops:
                continue;
            
            filtered_tokens += " " + stemmer.stem(tok)
            #carry out stemming specific to the language detected
            #filtered_tokens.add(stemmer.stem(tok))            
        
        #comment these out when you do not need to check if it works anymore
        #print("original tokens:", tokens,"\n")
        #print("filtered tokens:", filtered_tokens,"\n")
        
        #don't think having a column of the languages is really necessary but let's see
        return filtered_tokens, lang_prediction

In [9]:
#change the name of the property to tokens which is more appropriate
#training_set = training_set.rename(columns = {"tweetText" : "tokens"})

#make a new dataset from the original dataset transforming the tweet text into stemmed, simplified and filtered 
#tokens as handled by the TweetHandler class
def transform_data(arg):

    #copy the instance given so we don't change the original instance and can keep it in memory and reuse it 
    #if necessary
    dataset = copy.deepcopy(arg)
    th = TweetHandler()
    num_rows = dataset.label.size
    
    #the tweet text will be transformed into tokens so rename the column appropriately
    dataset = dataset.rename(columns = {"tweetText" : "tokens"})
    
    for i in range(num_rows):

        tweet = dataset["tokens"][i]
        label = dataset["label"][i]

        #disregard the humour information for now, simplify all the classes to two different classes
        if ("humor" in label) or ("fake" in label):
            label = 1
        else:
            label = 0
            
        #for testing
        #print("The old value of the row is:",dataset.loc[i],"\n")
        
        tokens, lang = th.parse_tweet(tweet)
        
        #change the type of tweetText from a string to a set of strings of the stemmed tokenized words
        dataset.loc[i] = tokens, label, lang
        
        #for testing
        #print("The new value of the row is:",dataset.loc[i],"\n\n")
    
    return dataset

In [10]:
#transform the data
simplified_training = transform_data(original_training)
simplified_testing = transform_data(original_testing)

In [11]:
#have a quick look to see if the tokens have been simplified how we want them to, which they have
simplified_training

,tokens,label,lang
0,acuerd pelicul el dia despues mañan me recuerd pas huracan sandy,1,es
1,milenagimon mir sandy ny tremend imag huracan parec dia independent real rt,1,es
2,buen fot huracan sandy recuerd pelicul dia independent id4 sandy,1,es
3,scari shit hurrican ny,1,en
4,my fave place world nyc hurrican sandi statueofliberti,1,en
...,...,...,...
14272,bobombdom slap tweetdeck pigfish,1,en
14273,new speci fish found brazil realli good photoshop what you think,1,en
14274,what call pigfish,1,en
14275,pigfish e dop pescecan pesc maial,1,it


In [12]:
#get an idea of how many of each language there are, we can see that it is predominantly english
simplified_training.lang.value_counts(normalize = True) * 100

en         76.598725
es          9.084542
unknown     7.508580
fr          1.505919
pt          1.155705
de          0.847517
it          0.707432
nl          0.644393
ar          0.560342
ru          0.420256
sv          0.294179
no          0.273167
da          0.217132
fi          0.091056
ro          0.049030
hu          0.042026
Name: lang, dtype: float64

In [13]:
simplified_training.label.value_counts()

1    9356
0    4921
Name: label, dtype: int64

In [14]:
simplified_testing[simplified_testing.lang == "en"]

,tokens,label,lang
0,kereeen rt shyman33 eclips iss,1,en
1,absolut beauti rt shyman33 eclips iss,1,en
2,shyman33 eclips iss 우주에서본 일식 wow amaz,1,en
3,eclips iss,1,en
8,dit dus rt the solar eclips seen intern space station solareclips iss space,1,en
...,...,...,...
3693,syria syrian hero boy rescu girl shootout,1,en
3695,böhmermann zdf neo varoufaki fake finger varoufak neo magazin royal mit jan b via youtub,1,en
3743,top stori jan böhmermann ist laut zdf fake see,1,en
3748,zdf neo lay jauch varoufaki stinkefing worth watch,1,en


In [15]:
#view the percentages of each language
simplified_testing.lang.value_counts(normalize = True) * 100

en         73.981358
unknown    14.993342
ar          5.166445
es          1.677763
de          1.091877
pt          0.958722
fr          0.852197
nl          0.612517
it          0.452730
fi          0.106525
sv          0.053262
ru          0.026631
ro          0.026631
Name: lang, dtype: float64

In [16]:
#check if there are any null values after simplifying, there aren't any
simplified_training.info()
print("\n")
simplified_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14277 entries, 0 to 14276
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tokens  14277 non-null  object
 1   label   14277 non-null  object
 2   lang    14277 non-null  object
dtypes: object(3)
memory usage: 334.7+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tokens  3755 non-null   object
 1   label   3755 non-null   object
 2   lang    3755 non-null   object
dtypes: object(3)
memory usage: 88.1+ KB


In [44]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

In [18]:
#class to convert the sparse matrix produced by the count vectorizer into a dense matrix in order for it 
#to be able to be used with different algorithms in a pipeline that require a dense matrix and not a sparse matrix
class Densifier():

    def fit(self, X, y=None, **kwargs):
        return self

    def transform(self, X, y=None, **kwarge):
        return X.todense()

In [19]:
def testNaiveBayes():
    
    #pipeline which always starts with a countvectorizer
    pipeline = Pipeline([
        ('cv', CountVectorizer()), 
        #('dt', Densifier()), 
        ('nb', MultinomialNB())
    ])

    #carry out a grid search for optimal parameter selection. Due to constraints of using my laptop this
    #can't be done that extensively, I will leave it to run overnight a few times
    clf = GridSearchCV(pipeline, {
        'cv__ngram_range': [(1,x) for x in range(9,10)],
        'cv__max_features' : [1000* i for i in range(8,12)]
        #,'nb__alpha' : [0.5 * i for i in range (5,7)]
    }, scoring='f1', verbose = 4) 
    #make sure we are focusing on maximizing the f1 score and not a different metric, add some verbosity so we can
    #see the progress of the grid search to get an idea of how much time it is taking
    
    #make sure the labels are treated as ints and not objects
    clf.fit(simplified_training.tokens,simplified_training.label.astype('int'))

In [20]:
def gridSearch(pipeline, params):
    
    #carry out a grid search for optimal parameter selection. Due to constraints of using my laptop this
    #can't be done that extensively, I will leave it to run overnight a few times
    
    #make n_jobs -1 so that all cores that are available are used which should hopefully make it quicker
    
    clf = GridSearchCV(pipeline, params, scoring = "f1", verbose = 6, n_jobs = -1)
    
    #make sure we are focusing on maximizing the f1 score and not a different metric, add some verbosity so we can
    #see the progress of the grid search to get an idea of how much time it is taking
    
    #make sure the labels are treated as ints and not objects
    clf.fit(simplified_training.tokens, simplified_training.label.astype("int"))
    return clf

In [39]:
#pipeline which always starts with a countvectorizer
mNaiveBayesPipeline = Pipeline([
    ('cv', CountVectorizer()), 
    #('dt', Densifier()), #used for converting from sparse matrix to dense matrix
    ('nb', MultinomialNB())
])

#the second lot of parameters to try
mNaiveBayesParams = {
    'cv__ngram_range': [(1,x) for x in range(14,18)],
    'cv__max_features' : [500 * i for i in range(2 * 12,2 * 17)],
    'nb__alpha' : [0.5 * i for i in range (0,6)]
}

#ignore this one for now
cNBpipeline = Pipeline([('cv', CountVectorizer()), ('cnb', ComplementNB())])
cNBparams = {
    'cv__ngram_range': [(1,x) for x in range(14,18)],
    'cv__max_features' : [1000* i for i in range(11,18)],
    'cnb__alpha' : [1 * i for i in range (0,9)]
}

#add: ('pca', PCA()),
lsvcPipeline = Pipeline([('cv', CountVectorizer()), ('df', Densifier()), ('pca', PCA()), ('lsvc', LinearSVC())])
lsvcParams = {
    'cv__ngram_range': [(1,x) for x in range(10,15)],
    'cv__max_features' : [1000* i for i in range(14,18)],
    'lsvc__C' : [1 * i for i in range(0,3)]
}

rForestPipeline = Pipeline([('cv', CountVectorizer()), ('rf', RandomForestClassifier())])
rForestParams = {
    'cv__ngram_range': [(1,x) for x in range(13,19)],
    'cv__max_features' : [1000* i for i in range(10,17)],
    'rf__n_estimators' : [10 * i for i in range(10,15)]
}


In [34]:
from sklearn.metrics import classification_report

In [35]:
def reportScores(clf):
    
    print("---- RESULTS ----","\n")
    print("The algorithm being optimised was:",clf.estimator.steps[-1])
    print("The best parameters found were:", clf.best_params_)
    y_test_true = simplified_testing.label
    y_test_predictions = clf.predict(simplified_testing.tokens)
    print("Score report:\n")
    #prevent the chance of any of the lists being treated as objects
    print(classification_report(y_test_true.astype('int'), y_test_predictions.astype('int')))

In [ ]:
#FIRST SEARCH: ngram 3-16, max features 5-16, alpha 0-9, RESULT: ngram: (1,15), features: 15000, alpha: 0
#so now try making the features and ngram even LARGER! got an accuracy of 81

#SECOND SEARCH: similar not much change, ngram 14-18, features 13-19, alpha 0-2
#RESULTS: features 14000, ngram 15, alpha 0
mNBResult = gridSearch(mNaiveBayesPipeline, mNaiveBayesParams)
reportScores(mNBResult)

In [ ]:
# best parameters found from alpha 0-9, max features 5k-16k and ngram (3,16): 
# alpha 2, max features 15000, ngram (1,15) with accuracy of 66. Not very good
cNBResult = gridSearch(cNBpipeline, cNBparams) 
reportScores(cNBResult)

In [ ]:
# 66 was okay... tried ngram 13-17, max features 13-17 and estimators 80-110 and best was 13000 features, ngram 1,15 and n estimators 110
#so try again with everything a bit higher
rForestResult = gridSearch(rForestPipeline, rForestParams)
reportScores(rForestResult)

In [ ]:
#parameters searched: ngram 10-12, max features 10-12, C 0-2, found: ngram 11, max features 12000
#introducing PCA made the computer die so don't use it again
lsvcResult = gridSearch(lsvcPipeline, lsvcParams)
reportScores(lsvcResult)

In [ ]:
# EXTRA ONES TO DO AFTERWARDS, KEEP ADDING THEM BELOW THEN COMMENT THE RESULTS UP ABOVE

In [31]:
rForestResult = gridSearch(rForestPipeline, rForestParams)
reportScores(rForestResult)

Fitting 5 folds for each of 210 candidates, totalling 1050 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 297 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 16.2min
[Parallel(n_jobs=-1)]: Done 597 tasks      | elapsed: 22.9min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 30.7min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed: 40.2min
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed: 42.6min finished


---- RESULTS ---- 

The algorithm being optimised was: ('rf', RandomForestClassifier())
The best parameters found were: {'cv__max_features': 12000, 'cv__ngram_range': (1, 15), 'rf__n_estimators': 100}
Score report:

              precision    recall  f1-score   support

           0       0.28      0.05      0.08      1209
           1       0.68      0.94      0.79      2546

    accuracy                           0.65      3755
   macro avg       0.48      0.49      0.43      3755
weighted avg       0.55      0.65      0.56      3755



In [32]:
lsvcResult = gridSearch(lsvcPipeline, lsvcParams)
reportScores(lsvcResult)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   56.7s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 297 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 11.0min finished


---- RESULTS ---- 

The algorithm being optimised was: ('lsvc', LinearSVC())
The best parameters found were: {'cv__max_features': 15000, 'cv__ngram_range': (1, 13), 'lsvc__C': 1}
Score report:

              precision    recall  f1-score   support

           0       0.64      0.80      0.71      1209
           1       0.89      0.78      0.84      2546

    accuracy                           0.79      3755
   macro avg       0.77      0.79      0.77      3755
weighted avg       0.81      0.79      0.79      3755



In [ ]:
mNBResult = gridSearch(mNaiveBayesPipeline, mNaiveBayesParams)
reportScores(mNBResult)

In [ ]:
#NOW TRY DOING THE SAME BUT USING TFIDF INSTEAD

mnbtPipeline = Pipeline([
    ('tf', TfidfVectorizer()), 
    #('dt', Densifier()), #used for converting from sparse matrix to dense matrix
    ('nb', MultinomialNB())
])

#the second lot of parameters to try
mnbtParams = {
    'tf__ngram_range': [(1,0.5 * x) for x in range(2 * 12,2 *17)],
    'tf__norm' : ["l1", "l2"],
    'nb__alpha' : [0.25 * i for i in range (0,6)]
}

#ignore this one for now
cnbtPipeline = Pipeline([('tf', TfidfVectorizer()), ('cnb', ComplementNB())])
cnbtParams = {
    'tf__ngram_range': [(1,x) for x in range(14,18)],
    'tf__norm' : ["l1", "l2"],
    'cnb__alpha' : [1 * i for i in range (0,9)]
}

#add: ('pca', PCA()),
lsvctPipeline = Pipeline([('tf', TfidfVectorizer()), ('df', Densifier()), ('lsvc', LinearSVC())])
lsvctParams = {
    'tf__ngram_range': [(1,x) for x in range(10,15)],
    'tf__norm' : ["l1", "l2"],
    'lsvc__C' : [1 * i for i in range(0,3)]
}

rftPipeline = Pipeline([('tf', TfidfVectorizer()), ('rf', RandomForestClassifier())])
rftParams = {
    'tf__ngram_range': [(1,x) for x in range(13,19)],
    'tf__norm' : ["l1", "l2"],
    'rf__n_estimators' : [10 * i for i in range(10,15)]
}


In [ ]:
#FIRST SEARCH: ngram 14-18, alpha 0-6, RESULT: alpha = 0.5, ngram = 14, norm = l2, accuracy = 87

#SECOND SEARCH: 
mnbtResult = gridSearch(mnbtPipeline, mnbtParams)
reportScores(mnbtResult)

In [ ]:
cnbtResult = gridSearch(cnbtPipeline, cnbtParams)
reportScores(cnbtResult)

In [ ]:
lsvctResult = gridSearch(lsvctPipeline, lsvctParams)
reportScores(lsvctResult)

In [ ]:
#FIRST SEARCH: 
rftResult = gridSearch(rftPipeline, rftParams)
reportScores(rftResult)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 14.6min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 39.7min
